In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from PMASfunc import *

In [2]:
def create_13_arcsec_minicube(wo_cube,wo_err,wo_header,brightest_pixel_x,brightest_pixel_y,box_size):
    mini_cube_data = wo_cube[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    mini_cube_err = wo_err[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    wo_header['CRPIX1'] = wo_header['CRPIX1'] - (brightest_pixel_x-box_size)
    wo_header['CRPIX2'] = wo_header['CRPIX2'] - (brightest_pixel_y-box_size)
    return mini_cube_data, mini_cube_err,wo_header

In [3]:
def scale_factor_spectrum(z,data,error,wo_wave):
    k = 1+z
    select1 = (wo_wave>5100*k) & (wo_wave<5130*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

def emp_fact(z,mini_cube,mini_err,wo_wave):
    true_err = np.zeros((np.shape(wo_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            k = 1+z
            select1 = (wo_wave>5100*k) & (wo_wave<5130*k)
   
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])  
            (spec,err) = (spectrum[select1],error[select1])
            (error_maincube,error_errcube) = (np.std(spec),np.mean(err))
            fact = error_maincube/error_errcube
            new_err = fact*error
            true_err[:,i,j] = new_err 
    return true_err
    

In [4]:
def algorithmic_script(obj,z,box_size=6,min_wave=4750,max_wave=5090,prefix_path_cube="/home/mainak/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/mainak/Downloads/Outflow_paper1/PMAS"):
    (wo_cube,wo_err,wo_wave,wo_header) = loadCube('%s/%s/%s.wo_absorption.fits'%(destination_path_cube,obj,obj)) 
    [x0,y0] = agn_loc(obj)
    (brightest_pixel_x,brightest_pixel_y) = (int(x0),int(y0))
    
    (mini_cube,mini_err,wo_header)=create_13_arcsec_minicube(wo_cube,wo_err,wo_header,brightest_pixel_x,brightest_pixel_y,box_size=6)
    true_err = emp_fact(z,mini_cube,mini_err,wo_wave)
    k = 1+z
    select = (wo_wave > min_wave*k) & (wo_wave < max_wave*k)  
    mini_header = wo_header
    mini_wave = wo_wave[select]
    mini_cube = mini_cube[select]
    mini_err = true_err[select]
    store_cube('%s/%s/%s.13_arcsec_minicube.fits'%(destination_path_cube,obj,obj),mini_cube,mini_wave,mini_err,mini_header)
    print ('%s'%(obj))

In [5]:
z = {"HE0853-0126":0.05981,"HE0949-0122":0.01993}
objs = z.keys()


for obj in objs:
    algorithmic_script(obj,z[obj])              


HE0853-0126
HE0949-0122
